In [11]:
import pandas as pd
import numpy as np
import requests
import os
from sklearn.model_selection import train_test_split
from collections import defaultdict
from tqdm import tqdm
from typing import Dict, List, Any

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Project/Side_Project/H&M_recommendation_system/data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Project/Side_Project/H&M_recommendation_system/data


In [3]:
path = '/content/gdrive/MyDrive/Project/Side_Project/H&M_recommendation_system/data'
files = os.listdir(path)

In [4]:
dfs = [pd.read_csv(file) for file in files]
df = dfs[0]

In [5]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'movieId', 'timestamp'], axis = 1)

,userId,rating,movie_id_idx
0,5,4.5,0
1,5,2.5,9
2,5,3.5,18
3,5,5.0,29
4,5,4.5,33
...,...,...,...
5392020,2704,5.0,1730
5392021,2704,3.0,1769
5392022,2704,4.0,1772
5392023,2704,4.0,1780


In [6]:
df = df[['userId', 'movie_id_idx', 'rating']]

In [7]:
# Information of Dataset
num_of_users = df['userId'].nunique()
num_of_movies = df['movie_id_idx'].nunique()
print('num_of_users:', num_of_users)
print('num_of_movies:', num_of_movies)

num_of_users: 10000
num_of_movies: 2000


In [8]:
# Split into train and test

train, test = train_test_split(df, test_size = 0.2)

In [17]:
# user_movie = {user_id : [movie1, movie2, ...]}
# movie_user = {movie_idx : [user1, user2, ...]}
# user_movie_rating = {(user, movie) : rating, (user2, movie2) : rating, ...}

def create_dictionary(target:List) -> Dict:
    result_dict = defaultdict(list)
    for v1, v2 in target:
        result_dict[v1].append(v2)
    return result_dict

In [ ]:
# [(user1, movie1), (rating)]
temp_data = list(zip(df['userId'], df['movie_id_idx'], df['rating']))
temp_dict = dict(((u, m), r) for u,m,r in temp_data)
temp_dict